In [19]:
import pandas as pd
import re
import os
from numpy.random import uniform as u
import glob

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#read one file from data to get insights of file's data
open('/content/gdrive/My Drive/Assignment2/data/2012-12-25','r').read()

'{"city": "kolkata", "code": "17749911", "title": "Ready to rock on New Year\'s Eve?", "text": "KOLKATA: The city is ready to party wild on New Year\'s Eve. With the closure deadline extended to 4am till January 15, star hotels and nightclubs are back to organizing gala parties and lavish dinners for revellers. Several are planning more entertainment events during the first week of the new year to take advantage of the late closure.At Underground in Hotel Hindustan International (HHI), a fairytale theme party will usher in 2013. Titled \'Ibiza\', the party will have fairies dancing to music and the club will be decorated with candies. At Virgose, the other nightclub at HHI, a Wild West theme party has been planned. The hotel is working on another entertainment event on December 29.\\"The deadline extension couldn\'t have come at a better time. Things were looking quite gloomy till last week. Now that we are allowed to stay open till 4 am, we might as well make full use of it. This also

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [8]:
!ls ../content

gdrive	sample_data  spark-3.2.0-bin-hadoop3.2	spark-3.2.0-bin-hadoop3.2.tgz


In [9]:
import json

In [10]:
# reading multiple files
mult_files = '/content/gdrive/My Drive/Assignment2/data*'

In [11]:
## 1. Read the data (all the files in the ‘data’ directory) using the function textFile
text = sc.textFile(mult_files)

In [12]:
text.collect()

Output hidden; open in https://colab.research.google.com to view.

In [13]:
##Take only the “text” part of each article and count the frequency of all the words (convert the text into lowercase)
textR = text.flatMap(lambda x:json.loads(x)['text'].lower().split())
mapR = textR.map(lambda x:(x,1))
word_countR = mapR.reduceByKey(lambda k,v:k+v)

In [15]:
word_countR.collect()

[('less', 8316),
 ('also', 150597),
 ('state', 88619),
 ('hurriedly', 185),
 ('children,', 2155),
 ('organize', 1569),
 ('india?buoyed', 1),
 ('handwritings', 2),
 ('"was', 101),
 ('&amp;', 5617),
 ('food,', 901),
 ('"controversy"', 2),
 ('facility', 3703),
 ('nodal', 831),
 ('records.source:', 4),
 ('annie', 121),
 ('tempted', 92),
 ('motto,', 11),
 ('members,', 1987),
 ('nominate', 251),
 ('covered,', 62),
 ('discipline', 683),
 ('kanpur,', 351),
 ('ladies', 680),
 ('grh', 78),
 ('boring', 359),
 ('persons.', 722),
 ("session's", 50),
 ('farms', 437),
 ('rajnagar', 37),
 ('blood', 5607),
 ('conventional', 719),
 ('jail.', 966),
 ('ministers', 4705),
 ('designer', 954),
 ('struggle.', 141),
 ('god.', 181),
 ('chamber', 1371),
 ('goacollege', 1),
 ('landscape', 351),
 ('frosting', 3),
 ('canaud', 1),
 ('(donald', 3),
 ('adding:', 178),
 ('limited,', 655),
 ("'khaleja'", 3),
 ('costly', 482),
 ('man."that\'s', 1),
 ('"superficial', 4),
 ("robbers'", 19),
 ('sunita', 559),
 ('coimbatore'

In [16]:
##Remove (Filter) any word whose frequency is less than 10
count_more_than_10_R = word_countR.filter(lambda x: x[1] >= 10)
count_more_than_10_R.collect()

[('less', 8316),
 ('also', 150597),
 ('state', 88619),
 ('hurriedly', 185),
 ('children,', 2155),
 ('organize', 1569),
 ('"was', 101),
 ('&amp;', 5617),
 ('food,', 901),
 ('facility', 3703),
 ('nodal', 831),
 ('annie', 121),
 ('tempted', 92),
 ('motto,', 11),
 ('members,', 1987),
 ('nominate', 251),
 ('covered,', 62),
 ('discipline', 683),
 ('kanpur,', 351),
 ('ladies', 680),
 ('grh', 78),
 ('boring', 359),
 ('persons.', 722),
 ("session's", 50),
 ('farms', 437),
 ('rajnagar', 37),
 ('blood', 5607),
 ('conventional', 719),
 ('jail.', 966),
 ('ministers', 4705),
 ('designer', 954),
 ('struggle.', 141),
 ('god.', 181),
 ('chamber', 1371),
 ('landscape', 351),
 ('adding:', 178),
 ('limited,', 655),
 ('costly', 482),
 ("robbers'", 19),
 ('sunita', 559),
 ('coimbatore', 2174),
 ('manabendra', 10),
 ('printers', 82),
 ('away,', 810),
 ('roasting', 11),
 ('mockery', 159),
 ('beneath', 258),
 ('reassurance', 26),
 ('age-old', 247),
 ('credit.', 126),
 ('botanical', 164),
 ('venue', 1757),
 ('p

In [17]:
##Total size of the output data (after the filtering)
total_size = count_more_than_10_R.count()
print('total size of output data after filtering: ', total_size)

total size of output data after filtering:  110620


In [18]:
##Frequency of the following words – congress, london, washington, football
given_word = ["congress", "london", "washington", "football"]
given_word_freq = count_more_than_10_R.filter(lambda x: x[0] in given_word)
frequencies = given_word_freq.collect()
print(frequencies)

[('congress', 28093), ('washington', 546), ('london', 2389), ('football', 1387)]


In [30]:
## The word with maximum frequency for each month (hint: to read only a month’s articles you can use ‘*’.
##E.g. for February ‘2012-02*’ represents all files starting with 2012-02,i.e. files belonging to Feb) [2 points]
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
for i in months:
  files = '/content/gdrive/My Drive/Assignment2/data/2012-'+i+'*'
  text = sc.textFile(files)
  textR = text.flatMap(lambda x:json.loads(x)['text'].lower().split())
  mapR = textR.map(lambda x:(x,1))
  word_countR = mapR.reduceByKey(lambda k,v:k+v)
  max_count = word_countR.map(lambda x:(x[1],x[0])).max()
  print('2012-'+i,max_count)



2012-01 (245413, 'the')
2012-02 (266950, 'the')
2012-03 (283107, 'the')
2012-04 (234165, 'the')
2012-05 (309021, 'the')
2012-06 (267383, 'the')
2012-07 (294923, 'the')
2012-08 (282393, 'the')
2012-09 (264449, 'the')
2012-10 (279876, 'the')
2012-11 (305296, 'the')
2012-12 (305414, 'the')


In [43]:
##List of words appeared on ‘2012-09-01’ but not on ‘2012-08-01’
def word_count(text):
  textR = text.flatMap(lambda x:json.loads(x)['text'].lower().split())
  mapR = textR.map(lambda x:(x,1))
  word_countR = mapR.reduceByKey(lambda k,v:k+v)
  return word_countR
file1 = '/content/gdrive/My Drive/Assignment2/data/2012-09-01*'
text1 = sc.textFile(file1)
sep_word_count = word_count(text1)
file2 = '/content/gdrive/My Drive/Assignment2/data/2012-08-01*'
text2 = sc.textFile(file2)
aug_word_count = word_count(text2)

print('unique_september_word:',sep_word_count.subtract(aug_word_count).collect())

unique_september_word: [('as', 908), ('naval', 6), ('command', 2), ('formally', 4), ('objectives,', 2), ('harness', 2), ('potential', 2), ('posting', 4), ('water', 114), ('kick', 8), ('start', 34), ('goa.', 4), ('bench', 40), ('friday', 224), ('remarks', 6), ('office-bearers', 2), ('record', 14), ('thunderous', 2), ('sharma,', 10), ('prism', 2), ('presence', 30), ('night.the', 2), ('poets', 4), ('dig', 4), ('everything,', 2), ('starting', 12), ('right', 42), ('relationship,', 2), ('ye', 2), ('only', 218), ('applauded', 2), ('poets,', 2), ('introduced', 12), ('even', 158), ('covered', 12), ('ceremony', 6), ('when', 306), ('saying', 32), ('raised', 34), ('felt', 12), ('pe', 2), ('further', 68), ('adarsh', 4), ('bihar', 18), ('us', 74), ('good', 50), ('power', 116), ('fails', 4), ('worst', 8), ('where', 122), ('failed', 40), ('enhanced', 2), ('things', 32), ('electricity', 20), ('officials', 116), ('conditions', 14), ('move', 48), ('lots', 6), ('hardship', 2), ('unsafe', 4), ('dark', 8), 

In [58]:
##Frequency of the word ‘monsoon’ for all months
monsoon_frequencies=[]
for i in months:
  files = '/content/gdrive/My Drive/Assignment2/data/2012-'+i+'*'
  text = sc.textFile(files)
  textR = text.flatMap(lambda x:json.loads(x)['text'].lower().split())
  mapR = textR.map(lambda x:(x,1))
  word_countR = mapR.reduceByKey(lambda k,v:k+v)
  monsoon_count = word_countR.filter(lambda x:x[0]=='monsoon').collect()
  monsoon_frequencies.append(monsoon_count)
  print('2012-'+i,monsoon_count)

2012-01 [('monsoon', 42)]
2012-02 [('monsoon', 52)]
2012-03 [('monsoon', 81)]
2012-04 [('monsoon', 121)]
2012-05 [('monsoon', 334)]
2012-06 [('monsoon', 934)]
2012-07 [('monsoon', 904)]
2012-08 [('monsoon', 505)]
2012-09 [('monsoon', 410)]
2012-10 [('monsoon', 252)]
2012-11 [('monsoon', 113)]
2012-12 [('monsoon', 59)]


In [59]:
## total monsoon word frequencies for all months
total_freq = sum([x[0][1] for x in monsoon_frequencies])
print('total monsoon word frequencies for all months: ', total_freq)


total monsoon word frequencies for all months:  3807
